<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [2]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-03-07 11:38:26--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-03-07 11:38:26--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  19.3MB/s    in 74s     

2024-03-07 11:39:40 (18.8 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [3]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1",
                                                  binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [77]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape,
          type(wv_embeddings), wv_embeddings["wait"])

float32 (200,) <class 'gensim.models.keyedvectors.KeyedVectors'> [ 2.1406817  -0.8892997  -1.0020239   1.7068193  -0.35036787  1.6860982
  0.53259635 -1.4178532  -2.7451923   0.20692238 -0.30007643  0.48497677
 -0.12604715  1.1446774  -3.530826    0.80622166 -0.5018453   0.23675618
 -0.71826655  0.1650759   1.0571383   1.5298465  -2.57817     1.788262
  4.105884   -0.3294636   2.6228     -1.6372417   0.5205855   0.47652048
  1.5095215  -0.07470538 -0.3605922  -2.093871   -2.0535798  -0.72039723
  0.26937887 -0.7100821  -1.3579862  -1.460941   -3.341229    1.1603973
 -2.1877348  -0.8973806   0.83869666  1.8041947  -1.8910159   2.3875246
  1.6102883   1.5937945   2.5459037   1.3553252  -1.5035651  -0.200593
  0.572839   -2.2874267  -0.8210639   0.05303868  1.3893535  -0.09278294
  1.9471496  -3.1448827   1.4168679   1.1914977  -0.5640521  -3.5474327
  2.0956972   1.9699324  -1.0212792   1.4276208  -2.1982565   4.096702
  2.247466   -1.4126004  -0.08287151  0.7099426   2.485224    2.12921

In [5]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [6]:
def cosine_sim(vector_1, vector_2):
    return np.array(vector_1 @ vector_2.T / norm(vector_1) / norm(vector_2))

In [7]:
# method most_simmilar
import numpy as np
from numpy.linalg import norm
top = np.zeros(len(wv_embeddings.index_to_key))
for i in range(len(wv_embeddings.index_to_key)):
  cos_dist = cosine_sim(wv_embeddings["dog"], wv_embeddings[i])
  top[i] = cos_dist

top = np.sort(top)
print(top[-6:-1])

[0.76079381 0.76212525 0.76238048 0.78808671 0.85641795]


In [8]:
print(cosine_sim(wv_embeddings["dog"], wv_embeddings["cat"]))
print(1 >= cosine_sim(wv_embeddings["dog"], wv_embeddings["cat"]) >= top[-6])
print("Cat не входит в топ 5 по близости к слову dog")

0.6852341
False
Cat не входит в топ 5 по близости к слову dog


In [9]:
for i in range(len(wv_embeddings.index_to_key)):
  if (round(top[-i], 7) == 0.6852341):
    print(f"Cat занимает {i - 1} место по близости к слову dog")
    break

Cat занимает 26 место по близости к слову dog


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [166]:
import numpy as np
import re
import spacy
from nltk.tokenize import WordPunctTokenizer
vector_size = 250
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
      tokens = re.findall('\w+', text.lower())
      tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens
                if token.lower() not in stop_words and not token.isdigit()]
      return tokens

tokenizer = MyTokenizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [143]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    i = 0
    vector = np.zeros(dim)
    for token in tokenizer(question):
      if token in embeddings.key_to_index:
        vector += embeddings[token]
        i += 1
    if i == 0 :
      i += 1
    return vector / i

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [159]:
print(round(question_to_vec("I love neural networks", wv_embeddings,
                            tokenizer.tokenize, 200)[2], 2))

-1.41


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

максимум Hits@47 - DCG@1 достигается для Hits@47, когда для всех вопросов дубликат стоит от 1 до 47 места, а для DCG@1 - когда дубликат для всех вопросов не стоит на первом месте => 1 - 0 = 1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [160]:
import math
DCG = 1 / math.log2(1 + 9) #предполагаю, что для одного вопроса
print(round(DCG, 1))

0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [161]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = 0
    for i in dup_ranks:
      hits_value += 1 if i <= k else 0
    return hits_value / len(dup_ranks)

In [162]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = 0
    for i in dup_ranks:
      dcg_value += 1 / math.log2(1 + i) if i <= k else 0
    return dcg_value / len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [24]:
import pandas as pd

In [33]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [34]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)),
                                            1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [1]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [35]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line)
    return data

Нам понадобиться только файл validation.

In [36]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [71]:
len(validation_data)

3760

Размер нескольких первых строк

In [101]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 54517
2 52818
3 54464
4 53724
5 52297


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [72]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    vec_question = question_to_vec(question, embeddings, tokenizer, dim)
    list_vec_candates = np.zeros((len(candidates), dim))
    for i, candidat in enumerate(candidates):
      list_vec_candates[i] = question_to_vec(candidat,
                                             embeddings,
                                             tokenizer,
                                             dim)
    distance = cosine_similarity(vec_question.reshape(1, -1),
                                 list_vec_candates)[0]
    ranked_candidates = sorted(enumerate(candidates),
                               key=lambda x: distance[x[0]], reverse=True)
    return ranked_candidates

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [41]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object',
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [57]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer.tokenize)
        print(ranks)
        print()

[0.47886281 0.54629098 0.16188453]
[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[0.45695476 0.47689124 0.16329035]
[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [126]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'), #скрыт
            (2, 'select2 not displaying search results'), #скрыт
            (1, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [58]:
from tqdm.notebook import tqdm

In [87]:
wv_ranking = []
max_validation_examples = 1100
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    parts = line.split('\t')
    q = parts[0]
    ex = parts[1:]
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer.tokenize)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [88]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k),
                                              k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.417 | Hits@   1: 0.417
DCG@   5: 0.505 | Hits@   5: 0.583
DCG@  10: 0.527 | Hits@  10: 0.653
DCG@ 100: 0.574 | Hits@ 100: 0.879
DCG@ 500: 0.586 | Hits@ 500: 0.975
DCG@1000: 0.589 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [99]:
train_data = read_corpus('./data/train.tsv')

'converting string to list\tConvert Google results object (pure js) to Python object\n'

In [124]:
train_data[1]

'Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?\tEvent handling for geometries in Three.js?\n'

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [153]:
words = []
for i in train_data:
  words.append(tokenizer.tokenize(i))


In [163]:
vector_size = 250

In [154]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words,
                 vector_size=vector_size,
                 sg=1,
                 min_count=10,
                 window=5,
                 epochs=5).wv

In [164]:
wv_ranking = []
max_validation_examples = 2000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    parts = line.split('\t')
    q = parts[0]
    ex = parts[1:]
    ranks = rank_candidates(q, ex,
                            embeddings_trained,
                            tokenizer.tokenize,
                            vector_size)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [165]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k),
                                              k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.526 | Hits@   1: 0.526
DCG@   5: 0.610 | Hits@   5: 0.680
DCG@  10: 0.632 | Hits@  10: 0.747
DCG@ 100: 0.665 | Hits@ 100: 0.905
DCG@ 500: 0.674 | Hits@ 500: 0.978
DCG@1000: 0.676 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1) Токенизация с исключением стоп слов, которые не несут в себе много информации о контексте, цифр, знаков препинания и приведения слова к начальной форме, дает небольшой прирост к метрике качества по сравнению с простой токенезацией по пробелам. Так происходит, т.к. мы уменьшаем мощность алфавита, на котором нам необходимо обучиться, и модель может построить лучший контекст для имеющихся данных, которые имеют большой вклад в контекст предложения.

2) Нормализация слов помогает, т.к. в нашей задаче слова с разными формами и окончаниями имеют почти один и тот же контекст, и мы позволяем модели лучше обучится на одном слове, а не 10, которые по сути несут в себе одинаковую информацию.

3) Модель, которая строила эмбеддинги по нашим тренировочным данным, справляется лучше, чем модель, которая обучалась на каких-то других, т.к. контекст слов, который присутствует в вопросах со StackOverflow, другой по сравнению со словами, которые мы используем в реальной жизни. Возможно, большей эффективности мы могли добиться, если бы дообучили изначальную модель под немного другой контекст.

4) Главной причиной, на мой взгляд, является то, что мы получали эмбеддинг предложения просто из средней суммы эмбеддингов всех слов, которые входят в это предложение

5) На мой взгяд, стоит взять обученную модель, дообучить ее на нашей выборке (на самом деле я не знаю работает ли это также эффективно для текста, как для сверточных нейросетей), использовать более продвинутый подход для нахождения эмбеддингов предложений и использовать нейросеть для предсказывания ранжирования, а не просто косинусово расстояние предложений
